In this section we focus on Image segmentation. 

We analyse the Cradic Magnetic resonance imaging data from the **Sunnybrook cardiac database**. Each sunnybrook data contains a 3d time series of a persons heart beat.  